In [3]:
# =============================================================================
# LANGKAH 1: INSTALASI PAKET WAJIB DI SETIAP SESI
# Blok ini harus ada di paling atas dan dijalankan setiap kali sesi dimulai.
# =============================================================================
!pip install -q scikit-learn==1.3.2
!pip install -q imbalanced-learn==0.11.0
!pip install -q Sastrawi



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\ACER\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\ACER\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\ACER\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [4]:
# =============================================================================
# ANALISIS SENTIMEN JSS (V21 - VERSI STABIL DENGAN PELABELAN BENAR)
# =============================================================================

# SECTION 1: IMPORT LIBRARIES
import pandas as pd
import numpy as np
import re
import emoji
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import joblib
import os
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# Konfigurasi awal
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")
pd.set_option('display.max_colwidth', 200)

# SECTION 2: TEXT PREPROCESSING CLASS
class TextPreprocessor:
    def __init__(self):
        self.stop_words = {
            'yang', 'dan', 'di', 'ke', 'dari', 'untuk', 'dengan', 'pada', 'dalam', 'atau', 'ini', 'itu', 'juga', 'akan',
            'sudah', 'telah', 'adalah', 'ada', 'pernah', 'sedang', 'masih', 'sering', 'selalu', 'kadang', 'biasanya', 'mungkin', 
            'segera', 'seperti', 'karena', 'kalau', 'jika', 'tapi', 'tetapi', 'namun', 'pas', 'serta', 'agar', 'supaya',
            'oleh', 'sehingga', 'tersebut', 'yakni', 'yaitu', 'pula', 'saya', 'kamu', 'dia', 'mereka', 'kami', 'kita', 
            'ku', 'mu', 'pun', 'kah', 'lah', 'harus', 'buka', 'daftar', 'tolong', 'login', 'masuk', 'pilih', 'kirim', 
            'isi', 'pakai', 'akses', 'lihat', 'liat', 'buat', 'mohon', 'banget', 'sekali', 'aja', 'saja', 'saat', 'waktu',
            'aplikasi', 'program', 'sistem', 'website', 'fitur', 'menu', 'akun', 'data', 'layanan', 'lapor', 'aduan', 
            'keluhan', 'informasi', 'info', 'verifikasi', 'update', 'jss', 'yogyakarta', 'masyarakat', 'min', 'admin',
            'notifikasi', 'notif', 'email', 'upload', 'file', 'password', 'username', 'nomor', 'telepon', 'otp', 'nik', 
            'saran', 'solusi', 'bintang', 'wae', 'dab', 'lurr', 'sih', 'dong', 'deh', 'kok', 'koq', 'mah', 'nya', 
            'lho', 'toh', 'gais', 'kak', 'gan', 'bro', 'halo', 'hallo', 'hai', 'wah', 'apa', 'kenapa', 'bagaimana', 
            'kapan', 'dimana', 'kpd', 'dll', 'dsb', 'yth', 'cctv', 'tv', 'hotspot','maps','chat','tema','ndes','nggih'
        }
        self.slang_word_dict = {
            'eroorrr': 'error', 'eror': 'error', 'blom': 'belum', 'blm': 'belum', 'durung': 'belum', 'ndak': 'tidak', 
            'ga': 'tidak', 'gak': 'tidak', 'nggak': 'tidak', 'tdk': 'tidak', 'g': 'tidak', 'gk': 'tidak', 'ra': 'tidak',
            'mboten': 'tidak', 'gaada': 'tidak ada', 'gabisa': 'tidak bisa', 'gakbisa': 'tidak bisa', 'bgt': 'banget', 
            'dgn': 'dengan', 'dg': 'dengan', 'krn': 'karena', 'utk': 'untuk', 'sdh': 'sudah', 'udh': 'sudah', 'dah': 'sudah',
            'wes': 'sudah', 'sy': 'saya', 'dr': 'dari', 'seko': 'dari', 'klo': 'kalau', 'kalo': 'kalau', 'skrg': 'sekarang', 
            'lg': 'lagi', 'bnyk': 'banyak', 'byk': 'banyak', 'jg': 'juga', 'bkn': 'bukan', 'td': 'tadi', 'trs': 'terus', 
            'trus': 'terus', 'gmn': 'bagaimana', 'gimana': 'bagaimana', 'pripun': 'bagaimana', 'kpn': 'kapan', 'tgl': 'tanggal', 
            'thn': 'tahun', 'bln': 'bulan', 'kpd': 'kepada', 'ttp': 'tetap', 'sbg': 'sebagai', 'pdhl': 'padahal', 'great': 'bagus',
            'bbrp': 'beberapa', 'hp': 'ponsel', 'tp': 'tapi', 'gpp': 'tidak apa-apa', 'sm': 'sama', 'lgsg': 'langsung', 'muter2': 'lambat',
            'cmn': 'cuma', 'cuman': 'hanya', 'app': 'aplikasi', 'apps': 'aplikasi', 'knp': 'kenapa', 'msh': 'masih', 'jozz': 'bagus',
            'tbtb': 'tiba-tiba', 'moga2': 'semoga', 'mhin': 'mohon', 'nomer': 'nomor', 'pasword': 'password', 'tlp': 'telepon',
            'responnya': 'respons', 'lemot': 'lambat', 'ngelag': 'lambat', 'mubeng': 'lambat', 'muter-muter': 'lambat', 
            'muter': 'lambat', 'crash': 'error', 'force close': 'error tutup paksa', 'fc': 'error tutup paksa', 'loading': 'memuat', 
            'instal': 'pasang', 'uninstal': 'copot pemasangan', 'register': 'daftar', 'upgrade': 'tingkatkan', 'pemkot': 'pemerintah kota', 
            'pemda': 'pemerintah daerah', 'dindukcapil': 'dinas kependudukan pencatatan sipil', 'capil': 'pencatatan sipil', 'ktp': 'kartu tanda penduduk', 
            'kk': 'kartu keluarga', 'akte': 'akta', 'npwp': 'nomor pokok wajib pajak', 'opd': 'organisasi perangkat daerah', 'jogja': 'yogyakarta', 
            'yogya': 'yogyakarta', 'dishub': 'dinas perhubungan', 'dinkes': 'dinas kesehatan', 'suwun': 'terima kasih', 'okokok': 'oke',
            'matur nuwun': 'terima kasih', 'maturnuwun': 'terima kasih', 'nuwun': 'terima kasih', 'ngeten': 'begini', 'lelet': 'lambat',
            'angel': 'sulit', 'mumet': 'pusing', 'kesuwen': 'terlalu lama', 'suwe': 'lama', 'jan': 'sungguh', 'tenan': 'sungguh', 
            'malah': 'justru', 'iki': 'ini', 'opo': 'apa', 'iso': 'bisa', 'gawe': 'buat', 'ndelok': 'lihat', 'mantap': 'bagus', 
            'keren': 'bagus', 'good': 'bagus', 'nice': 'bagus', 'ok': 'oke', 'oke': 'oke', 'jos': 'bagus', 'joss': 'bagus', 
            'josss': 'bagus', 'mantab': 'bagus', 'mantuulll': 'bagus', 'mantabbb': 'bagus', 'apik': 'bagus', 'sae': 'bagus', 'istimewaa': 'bagus',
            'sip': 'bagus', 'siip': 'bagus', 'sippppppppppp': 'bagus', 'tjakep': 'bagus', 'mantul': 'bagus', 'istimewa': 'bagus', 
            'top': 'bagus', 'the best': 'terbaik', 'best': 'terbaik', 'dabest': 'terbaik', 'luarbiasa': 'luar biasa', 'force close': 'error',
            'bermanfaat': 'manfaat', 'membantu': 'bantu', 'helpful': 'bantu', 'sangat bantu': 'bantu', 'sangat membantu': 'bantu',
            'susah': 'sulit', 'dpersulit': 'sulit', 'ribet': 'sulit', 'bertele-tele': 'sulit', 'berbelit': 'sulit', 'payah': 'buruk', 
            'ancur': 'hancur', 'parah': 'sangat buruk', 'elek': 'jelek', 'bosok': 'busuk', 'gajelas': 'tidak jelas', 'ngak': 'tidak',
            'ga jelas': 'tidak jelas', 'zonk': 'gagal', 'thanks': 'terima kasih', 'makasih': 'terima kasih', 'mksh': 'terima kasih', 
            'trims': 'terima kasih', 'tks': 'terima kasih', 'trimakasih': 'terima kasih', 'diperbaiki': 'perbaiki', 'ditingkatkan': 'tingkat',
            'ookkee' : 'oke', 'oke oke': 'oke', 'marai': 'bikin', 'mubeng minger': 'lambat','pekok': 'bodoh','kliteh': 'kriminalitas', 
            'rong': 'belum', 'nyobo': 'coba','aing': 'saya','bolan-baleni': 'berulang kali','nglebokke': 'memasukkan','bagussss': 'bagus',
            'jempoooooooollllll': 'bagus', 'jempooll': 'bagus', 'verygood': 'sangat bagus', 'goodluck': 'semoga berhasil', "klk": "kalo",
            'quick respon': 'respons cepat', 'optimal': 'maksimal', 'baguss': 'bagus', "joshh": 'bagus', "dadi": "jadi", 'b aja': 'biasa saja'
        }
        factory = StemmerFactory()
        self.stemmer = factory.create_stemmer()

    def normalize_elongated_words(self, text):
        return re.sub(r'(.)\1{2,}', r'\1\1', text)

    def convert_emojis_to_words(self, text):
        emoji_mapping = {
            '👍': ' bagus ', '❤️': ' suka ', '😊': ' senang ', '👌': ' oke ', '✅': ' setuju ', '✨': ' bagus sekali ', '⭐': ' bintang ', '💯': ' sempurna ', '👏': ' bagus ', '🙏': ' terima kasih ',
            '👎': ' jelek ', '😡': ' marah ', '😠': ' marah ', '😤': ' kesal ', '😭': ' kecewa menangis ', '😢': ' sedih ', '😞': ' kecewa ', '💔': ' kecewa ', '💩': ' jelek sekali ', '❌': ' salah '
        }
        for emoji_char, meaning in emoji_mapping.items():
            text = text.replace(emoji_char, meaning)
        return emoji.demojize(text, delimiters=(" ", " ")).replace("_", " ")
        
    def _handle_negation(self, text):
        negation_words = {'tidak', 'bukan', 'jangan', 'kurang', 'belum'}
        words = text.split()
        processed_words = []
        i = 0
        while i < len(words):
            if words[i] in negation_words and i + 1 < len(words):
                processed_words.append(words[i] + '_' + words[i+1])
                i += 2
            else:
                processed_words.append(words[i])
                i += 1
        return ' '.join(processed_words)

    def preprocess_text(self, text):
        if pd.isna(text): return ""
        text = str(text).lower()
        text = self.convert_emojis_to_words(text)
        text = self.normalize_elongated_words(text)
    
        # --- PERBAIKAN URUTAN ---
        # 1. Hapus URL dan Tanda Baca DULU agar teks bersih
        text = re.sub(r'http[s]?://\S+|www\.\S+', ' ', text)
        text = re.sub(r'[^a-zA-Z\s_]', ' ', text)
        text = ' '.join(text.split())
    
        # 2. BARU lakukan normalisasi slang pada kata-kata yang sudah bersih
        words = text.split()
        normalized_words = [self.slang_word_dict.get(word, word) for word in words]
        text = ' '.join(normalized_words)
        
        # 3. Lanjutkan dengan stemming, negasi, dan stopword
        text = self.stemmer.stem(text)
        text = self._handle_negation(text)
        
        words = text.split()
        words = [word for word in words if word not in self.stop_words]
        return ' '.join(words)

# SECTION 2.5: FUNGSI PELABELAN OTOMATIS
def label_review_enhanced(row):
    review_text = str(row['Ulasan']).lower()
    rating = row['Rating']
    negative_keywords = ['kecewa', 'sulit', 'lambat', 'error', 'tidak bisa', 'berbelit', 'susah', 'buruk', 'parah', 'gagal', 'eror', 'masalah', 'kendala', 'tidak muncul', 'ribet', 'pusing', 'jelek', 'busuk',]
    if rating <= 2: return 'negatif'
    if rating == 5: return 'positif'
    if rating == 4:
        return 'negatif' if any(keyword in review_text for keyword in negative_keywords) else 'positif'
    if rating == 3:
        if any(keyword in review_text for keyword in negative_keywords): return 'negatif'
        return 'netral'
    return 'netral'

# SECTION 2.6: FUNGSI ANALISIS DATA-DRIVEN
def analyze_word_sentiment_distribution(df, threshold=0.70):
    print("\n" + "="*80)
    print("🔎 ANALISIS DISTRIBUSI SENTIMEN UNTUK ULASAN SATU KATA")
    print("="*80)
    
    single_word_reviews = df[df['word_count_cleaned'] == 1]
    
    if single_word_reviews.empty:
        print("Tidak ditemukan ulasan satu kata untuk dianalisis.")
        return set()

    distribution = single_word_reviews.groupby('cleaned_text')['sentimen'].value_counts(normalize=True)
    
    dist_df = distribution.unstack(level=-1).fillna(0)
    dist_df['count'] = single_word_reviews['cleaned_text'].value_counts()
    dist_df = dist_df.sort_values(by='count', ascending=False)
    
    print("Menampilkan distribusi untuk kata yang muncul lebih dari 2 kali:")
    print(dist_df[dist_df['count'] > 2].to_string(float_format="{:.2f}".format))
    
    strong_sentiment_words = set()
    for word, row in dist_df.iterrows():
        is_strong_positive = row.get('positif', 0) > threshold
        is_strong_negative = row.get('negatif', 0) > threshold
        
        if is_strong_positive or is_strong_negative:
            strong_sentiment_words.add(word)
            
    print(f"\n✅ Ditemukan {len(strong_sentiment_words)} kata sentimen kuat (threshold > {threshold*100}%)")
    print(f"   Kata-kata tersebut: {sorted(list(strong_sentiment_words))}")
    print("="*80 + "\n")
    
    return strong_sentiment_words

# SECTION 3: KELAS UTAMA SENTIMENT ANALYZER
class SentimentAnalyzerLR:
    def __init__(self):
        self.preprocessor = TextPreprocessor()
        self.pipeline = None
        self.is_trained = False
        self.pipeline = Pipeline([
            ('tfidf', TfidfVectorizer(max_features=5000, sublinear_tf=True)),
            ('classifier', LogisticRegression(random_state=42, max_iter=2000, class_weight='balanced'))
        ])

    def _analyze_error_reason(self, cleaned_text):
        pos_keywords = {'bagus', 'bantu', 'baik', 'puas', 'keren', 'hebat', 'manfaat', 'suka', 'senang', 'tarik'}
        neg_keywords = {'jelek', 'sulit', 'kecewa', 'lambat', 'error', 'buruk', 'parah', 'gagal', 'pusing'}
        words = set(cleaned_text.split())
        has_pos = any(word in pos_keywords for word in words)
        has_neg = any(word in neg_keywords for word in words)
        if has_pos and has_neg: return "Sinyal Sentimen Bertentangan"
        if any('_' in word for word in words): return "Konteks Negasi/Penguat Sulit"
        if not has_pos and not has_neg: return "Tidak Ada Kata Kunci Kuat"
        return "Lain-lain (Konteks/Sarkasme)"

    def train_and_evaluate(self, df_full, text_column='Ulasan', target_column='sentimen'):
        if 'cleaned_text' not in df_full.columns:
            df_full['cleaned_text'] = df_full[text_column].apply(self.preprocessor.preprocess_text)
        else:
            print("\n✔️ Kolom 'cleaned_text' sudah ada, melewati langkah preprocessing.")
        
        # Filter out rows with empty cleaned_text before splitting
        df_full = df_full[df_full['cleaned_text'].str.strip() != ''].copy()

        df_train, df_test = train_test_split(df_full, test_size=0.25, random_state=42, stratify=df_full[target_column])
        print(f"\nPartitioning Data: {len(df_train)} untuk training, {len(df_test)} untuk evaluasi.")

        print("\n🚀 Memulai Training Model dengan Pencarian Parameter Mendalam...")
        X_train, y_train = df_train['cleaned_text'], df_train[target_column]
        
        param_grid = {
            'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
            'tfidf__min_df': [2, 3],
            'classifier__C': [1, 10, 50, 100]
        }
        
        grid_search = GridSearchCV(self.pipeline, param_grid, cv=5, scoring='f1_macro', n_jobs=-1, verbose=2)
        grid_search.fit(X_train, y_train)
        self.pipeline = grid_search.best_estimator_
        self.is_trained = True
        print(f"✅ Training Selesai! Parameter terbaik: {grid_search.best_params_}")

        print("\n📈 HASIL EVALUASI MODEL:")
        X_test, y_test = df_test['cleaned_text'], df_test[target_column]
        y_pred = self.pipeline.predict(X_test)
        
        report_text = classification_report(y_test, y_pred, zero_division=0)
        print(report_text)
        
        self.save_results(report_text, grid_search)
        self.plot_and_save_confusion_matrix(y_test, y_pred)
        self.show_misclassified_data(df_test, y_pred)
        self.show_top_features()

    def save_results(self, report_text, grid_search):
        print("\n💾 Menyimpan hasil lengkap Grid Search...")
        results_df = pd.DataFrame(grid_search.cv_results_)
        results_df.sort_values(by='rank_test_score', inplace=True)
        results_df.to_csv('grid_search_results.csv', index=False)
        print("   -> Hasil disimpan ke 'grid_search_results.csv'")
        print("\n💾 Menyimpan laporan klasifikasi...")
        with open('classification_report.txt', 'w', encoding='utf-8') as f:
            f.write("="*50 + "\nLaporan Klasifikasi\n" + "="*50 + "\n" + report_text)
        print("   -> Laporan disimpan ke 'classification_report.txt'")

    def plot_and_save_confusion_matrix(self, y_true, y_pred):
        print("\n🖼️  Membuat, menyimpan, dan menampilkan confusion matrix...")
        labels = sorted(list(set(y_true) | set(y_pred)))
        cm = confusion_matrix(y_true, y_pred, labels=labels)
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
        plt.title('Confusion Matrix', fontsize=16)
        plt.xlabel('Prediksi', fontsize=12)
        plt.ylabel('Aktual', fontsize=12)
        plt.savefig('confusion_matrix.png', dpi=300, bbox_inches='tight')
        print("   -> Confusion matrix berhasil disimpan ke 'confusion_matrix.png'")
        plt.show()

    def show_misclassified_data(self, df_test, y_pred):
        print("\n" + "="*80)
        print("🕵️  ANALISIS KESALAHAN KLASIFIKASI (MISCLASSIFIED DATA)")
        print("="*80)
        misclassified_mask = df_test['sentimen'] != y_pred
        if not misclassified_mask.any():
            print("✅ Tidak ada data yang salah diklasifikasikan. Model sempurna!")
            print("="*80 + "\n")
            return
        misclassified_data = df_test[misclassified_mask].copy()
        misclassified_data['prediksi'] = y_pred[misclassified_mask]
        misclassified_data['analisis'] = misclassified_data['cleaned_text'].apply(self._analyze_error_reason)
        def print_group(title, df_group):
            print("\n" + "-"*80)
            print(f"📄 {title.upper()} (Total: {len(df_group)})")
            print("-"*80)
            if df_group.empty:
                print("Tidak ada kesalahan untuk kategori ini.")
                return
            for index, row in df_group.head(10).iterrows():
                print(f"   Teks Asli        : {row['Ulasan']}")
                print(f"   Teks Olahan      : {row['cleaned_text']}")
                print(f"   Label Seharusnya : {row['sentimen'].upper()}")
                print(f"   Prediksi Model   : {row['prediksi'].upper()}")
                print(f"   Analisis         : {row['analisis']}")
                print("-" * 40)
        pos_as_neg = misclassified_data[(misclassified_data['sentimen'] == 'positif') & (misclassified_data['prediksi'] == 'negatif')]
        neg_as_pos = misclassified_data[(misclassified_data['sentimen'] == 'negatif') & (misclassified_data['prediksi'] == 'positif')]
        sent_as_neu = misclassified_data[(misclassified_data['sentimen'] != 'netral') & (misclassified_data['prediksi'] == 'netral')]
        neu_as_sent = misclassified_data[(misclassified_data['sentimen'] == 'netral') & (misclassified_data['prediksi'] != 'netral')]
        print_group("Kesalahan Tipe 1: Sentimen POSITIF diprediksi NEGATIF", pos_as_neg)
        print_group("Kesalahan Tipe 2: Sentimen NEGATIF diprediksi POSITIF", neg_as_pos)
        print_group("Kesalahan Tipe 3: Sentimen Jelas (Pos/Neg) diprediksi NETRAL", sent_as_neu)
        print_group("Kesalahan Tipe 4: NETRAL diprediksi sebagai Sentimen Jelas", neu_as_sent)
        print("\n" + "="*80 + "\n")

    def show_top_features(self, n=15):
        try:
            tfidf_vectorizer = self.pipeline.named_steps['tfidf']
            classifier = self.pipeline.named_steps['classifier']
            feature_names = np.array(tfidf_vectorizer.get_feature_names_out())
            print("="*80)
            print(f"🔑 KATA KUNCI UTAMA PER SENTIMEN (TOP {n})")
            print("="*80)
            for i, class_label in enumerate(classifier.classes_):
                coefs = classifier.coef_[i]
                top_coef_indices = coefs.argsort()[-n:][::-1]
                top_words = feature_names[top_coef_indices]
                top_coefs = coefs[top_coef_indices]
                print(f"\n--- Sentimen: {class_label.upper()} ---")
                for word, coef in zip(top_words, top_coefs):
                    print(f"{word: <20} (Bobot: {coef:.3f})")
            print("="*80 + "\n")
        except Exception as e:
            print(f"Gagal menampilkan top features: {e}")

# SECTION 4: FUNGSI UTAMA UNTUK MENJALANKAN ALUR KERJA
def run_workflow(file_path, sheet_names, text_column='Ulasan'):
    print("="*80 + "\n🚀 ANALISIS SENTIMEN (V21 - Versi Stabil) 🚀\n" + "="*80)
    try:
        all_df = [pd.read_excel(file_path, sheet_name=s, header=None, names=['Ulasan', 'Rating']) for s in sheet_names]
        combined_df = pd.concat(all_df, ignore_index=True)
        combined_df.dropna(subset=['Ulasan', 'Rating'], inplace=True)
        combined_df['Rating'] = pd.to_numeric(combined_df['Rating'], errors='coerce')
        combined_df.dropna(subset=['Rating'], inplace=True)
        combined_df['Rating'] = combined_df['Rating'].astype(int)
        
        print("\n🤖 Menerapkan pelabelan otomatis awal (berdasarkan rating)...")
        combined_df['sentimen'] = combined_df.apply(label_review_enhanced, axis=1)
        print("Distribusi sentimen awal:")
        print(combined_df['sentimen'].value_counts())

        print("\n✨ Menyempurnakan pelabelan berbasis data...")
        temp_preprocessor = TextPreprocessor()
        combined_df['cleaned_text'] = combined_df[text_column].apply(temp_preprocessor.preprocess_text)
        combined_df['word_count_cleaned'] = combined_df['cleaned_text'].apply(lambda x: len(x.split()))
        
        strong_sentiment_words = analyze_word_sentiment_distribution(combined_df, threshold=0.70)
        
        condition_to_relabel = (combined_df['word_count_cleaned'] <= 1) & \
                               (~combined_df['cleaned_text'].isin(strong_sentiment_words))
        
        original_neutral_count = (combined_df['sentimen'] == 'netral').sum()
        combined_df.loc[condition_to_relabel, 'sentimen'] = 'netral'
        new_neutral_count = (combined_df['sentimen'] == 'netral').sum()
        
        print(f"✅ Pelabelan ulang selesai. {new_neutral_count - original_neutral_count} ulasan diidentifikasi sebagai netral.")

        print("\n📊 Distribusi sentimen final (setelah penyempurnaan):")
        print(combined_df['sentimen'].value_counts())

        analyzer = SentimentAnalyzerLR()
        analyzer.train_and_evaluate(combined_df, text_column='cleaned_text', target_column='sentimen') 
        
        model_filename = 'jss_sentiment_model_v21.pkl'
        print(f"\n💾 Menyimpan Model Final ke '{model_filename}'...")
        joblib.dump(analyzer, model_filename)
        print(f"   -> Model berhasil disimpan ke '{model_filename}'.")

    except Exception as e:
        print(f"❌ Terjadi kesalahan pada alur kerja utama: {e}")
        import traceback
        traceback.print_exc()

# SECTION 5: TITIK MASUK EKSEKUSI PROGRAM
if __name__ == "__main__":
    DATA_FILE_PATH = '/kaggle/input/jss-v5/Ulasan_JSSv5.xlsx'
    SHEET_NAMES_TO_READ = ['Positif', 'Negatif', 'Netral']
    
    if os.path.exists(DATA_FILE_PATH):
        run_workflow(file_path=DATA_FILE_PATH, sheet_names=SHEET_NAMES_TO_READ)
    else:
        try:
            df_pos = pd.read_csv('Ulasan_JSS.xlsx - Positif.csv', header=None, names=['Ulasan', 'Rating'])
            df_neg = pd.read_csv('Ulasan_JSS.xlsx - Negatif.csv', header=None, names=['Ulasan', 'Rating'])
            df_net = pd.read_csv('Ulasan_JSS.xlsx - Netral.csv', header=None, names=['Ulasan', 'Rating'])
            
            with pd.ExcelWriter('Ulasan_JSS.xlsx') as writer:
                df_pos.to_excel(writer, sheet_name='Positif', index=False, header=False)
                df_neg.to_excel(writer, sheet_name='Negatif', index=False, header=False)
                df_net.to_excel(writer, sheet_name='Netral', index=False, header=False)
            
            print("💡 File Excel tidak ditemukan, file CSV dibaca dan file Excel sementara dibuat.")
            run_workflow(file_path='Ulasan_JSS.xlsx', sheet_names=SHEET_NAMES_TO_READ)
            
        except FileNotFoundError:
            print("❌ ERROR: File Excel maupun CSV tidak ditemukan.")

ModuleNotFoundError: No module named 'emoji'